In [4]:
import scrapy
import pandas as pd
from readability.readability import Document

In [5]:
news_sites = 'csv_files/news_sites/btc_news_sites.csv'

In [6]:
sites = pd.read_csv(news_sites)
sites

,Source,URL
0,FT,https://www.ft.com/search?q=Bitcoin&sort=date
1,WSJ,https://www.wsj.com/search?query=Bitcoin&mod=s...
2,CNBC,https://www.cnbc.com/search/?query=Bitcoin&qse...
3,Bloomberg,https://www.bloomberg.com/search?query=Bitcoin
4,Reuters,https://www.reuters.com/site-search/?query=Bit...
5,Forbes,https://www.forbes.com/search/?sort=recent&q=b...
6,ZeroHedge,https://www.zerohedge.com/search-content?qTitl...
7,Seeking Alpha News,https://seekingalpha.com/symbol/BTC-USD/news
8,Seeking Alpha Analysis,https://seekingalpha.com/symbol/BTC-USD/analysis
9,Business Insider,https://markets.businessinsider.com/currencies...


In [ ]:
# Spider 1 
# Articles.py which scrape article links
# imports
from scrapy.http import Request

class ArticlesSpider(scrapy.Spider):
 name = 'Articles'
 allowed_domains = ['ft.com']
 start_urls = ['https://www.ft.com/']

def start_requests(self):
  # Hardcoded URL that contains TURKEY related subjects
  url="https://www.ft.com/search?q=Bitcoin&sort=date"
  link_urls = [url.format(i) for i in range(0,500)]

  # Loops through 500 pages to get the article links
  for link_url in link_urls:
    print(link_url)

      # Request to get the HTML content
      request=Request(link_url, cookies={'store_language':'en'}, 
      callback=self.parse_main_pages)
  yield request
  def parse_main_pages(self,response):
  item=TrtworldItem()

  # Gets HTML content where the article links are stored
    content=response.xpath('//div[@id="items"]//div[@class="article- 
    meta"]')
    
  # Loops through the each and every article link in HTML 'content'
    for article_link in content.xpath('.//a'):
  # Extracts the href info of the link to store in scrapy item
    item['article_url'] =    
    article_link.xpath('.//@href').extract_first()
  item['article_url'] =   
    "https://www.trtworld.com"+item['article_url']
  yield(item)
  def parse(self, response):
    pass

In [ ]:
'''
headline_scraper.py
A simple scrapy spider to collect web page titles
'''
news_sites = 'csv_files/news_sites/btc_news_sites.csv'


class HeadlineSpider(scrapy.Spider):
    name = "headline_spider"
    start_urls = pd.read_csv(news_sites).url.tolist()

    def parse(self, response):
        doc = Document(response.text)
        yield {
            'short_title': doc.short_title(),
            'full_title': doc.title(),
            'url': response.url
        }

In [ ]:
import scrapy

class FT_Spider(scrapy.Spider):
    #name of the spider
    name = 'ft'

    #list of allowed domains
    allowed_domains = ['ft.com']

    #starting url for scraping
    start_urls = ['https://www.ft.com/search/']

    #setting the location of the output csv file
    custom_settings = {
        'FEED_URI' : 'tmp/ft.csv'
    }

    def parse(self, response):
        #Remove XML namespaces
        response.selector.remove_namespaces()
    
        #Extract article information
        titles = response.xpath('//item/title/text()').extract()
        authors = response.xpath('//item/creator/text()').extract()
        dates = response.xpath('//item/pubDate/text()').extract()
        links = response.xpath('//item/link/text()').extract()

        for item in zip(titles,authors,dates,links):
            scraped_info = {
                'title' : item[0],
                'author' : item[1],
                'publish_date' : item[2],
                'link' : item[3]
            }

            yield scraped_info